In [9]:
%reset -f

import sys
import os
import bz2
import csv
import pandas as pd
import ast
import ujson
import numpy as np

dirpath = "/Volumes/platomo data/Projekte/008 BASt Fussverkehrsaufkommen/Ground Truth/Zählstellen_fertig/C1/Ost_OTC01/"

## Import otevents

In [14]:
with bz2.open('/Volumes/platomo data/Projekte/008 BASt Fussverkehrsaufkommen/Ground Truth/Zählstellen_fertig/C2/West_OTC28/C2_West_OTC28.events.otevents_alt', "rt", encoding="UTF-8") as file:
    dictfile = ujson.load(file)
    
    # Convert to DataFrame
    EVENTS = pd.DataFrame.from_dict(dictfile['event_list'])

    print(EVENTS[EVENTS['frame_number'] == 1111])

     road_user_id road_user_type    hostname                  occurrence  \
1037           11     pedestrian  otcamera28  2023-09-20 07:00:55.481130   

      frame_number section_id                       event_coordinate  \
1037          1111         11  [531.9104614257812, 269.136474609375]   

         event_type                          direction_vector  \
1037  section-enter  [-27.75592041015625, 1.3209899902343523]   

                                       video_name  
1037  otcamera28_FR20_2023-09-20_07-00-00_Ost.mp4  


In [16]:
with bz2.open('/Volumes/platomo data/Projekte/008 BASt Fussverkehrsaufkommen/Ground Truth/Zählstellen_fertig/C2/West_OTC28/C2_West_OTC28_neu.otevents', "rt", encoding="UTF-8") as file:
    dictfile = ujson.load(file)
    
    # Convert to DataFrame
    EVENTS = pd.DataFrame.from_dict(dictfile['event_list'])

    print(EVENTS[EVENTS['frame_number'] == 1111])

         road_user_id road_user_type    hostname                  occurrence  \
487  legacy#legacy#11            car  otcamera28  2023-09-20 07:00:55.481130   

     frame_number section_id                       event_coordinate  \
487          1111         11  [531.9104614257812, 269.136474609375]   

        event_type                          direction_vector  \
487  section-enter  [-27.75592041015625, 1.3209899902343523]   

                                      video_name  
487  otcamera28_FR20_2023-09-20_07-00-00_Ost.mp4  


In [13]:
def events_to_df(filepath, key = 'event_list'):
    # Open otevents-file
    with bz2.open(filepath, "rt", encoding="UTF-8") as file:
        dictfile = ujson.load(file)
    
    # Convert to DataFrame
    EVENTS = pd.DataFrame.from_dict(dictfile[key])

    return EVENTS

OTAEvents = events_to_df(dirpath + 'C1_Ost_OTC01.events.otevents')
GTEvents1 = events_to_df(dirpath + 'OTCamera01_FR20_2023-09-19_17-00-00_Sued.otgtevents')
GTEvents2 = events_to_df(dirpath + 'OTCamera01_FR20_2023-09-19_21-00-00_Sued.otgtevents')
GTEvents3 = events_to_df(dirpath + 'OTCamera01_FR20_2023-09-20_07-00-00_Sued.otgtevents')
GTEvents4 = events_to_df(dirpath + 'OTCamera01_FR20_2023-09-20_13-00-00_Sued.otgtevents')

# Nur Fußgaenger
# OTAEvents = OTAEvents[OTAEvents['road_user_type'] == 'pedestrian'].reset_index(drop=True)

# Nur Events mit vergebener section
OTAEvents = OTAEvents[OTAEvents['section_id'].isin(['9','10'])].reset_index(drop=True)


GTEvents = pd.concat([GTEvents1, GTEvents2, GTEvents3, GTEvents4])
del GTEvents1, GTEvents2, GTEvents3, GTEvents4

GTEvents['datetime'] = pd.to_datetime(GTEvents['occurrence'] + 7200, unit='s') #120min fehlen aus irgendeinem Grund



## Auswertung

In [23]:
OTAEvents['occurrence'] = pd.to_datetime(OTAEvents['occurrence'])
OTAEvents['Zeitspanne'] = OTAEvents['occurrence'].dt.floor('15min')
GTEvents['Zeitspanne'] = GTEvents['datetime'].dt.floor('15min')

OTACounts = pd.DataFrame(OTAEvents.groupby(['Zeitspanne', 'section_id', 'road_user_type'])['road_user_id'].count()).rename(columns = {'road_user_id': 'OTAnalytics'})
GTCounts = pd.DataFrame(GTEvents.groupby(['Zeitspanne', 'section_id', 'road_user_type'])['road_user_id'].count()).rename(columns = {'road_user_id': 'GroundTruth'})
# OTACounts

print(OTACounts)
print(GTCounts)

Counts = pd.merge(OTACounts, GTCounts)

                                               count
Zeitspanne          section_id road_user_type       
2023-09-19 17:00:00 10         bicyclist           3
                               pedestrian         33
                    9          bicyclist           3
                               pedestrian         26
2023-09-19 21:00:00 10         pedestrian          8
                    9          bicyclist           1
                               bus                 1
                               pedestrian          6
2023-09-20 07:00:00 10         car                 2
                               pedestrian         17
                    9          car                 1
                               pedestrian         21
2023-09-20 13:00:00 10         bicyclist           1
                               car                 3
                               pedestrian         31
                               scooter_driver      1
                    9          bicyclist      